In [4]:
import pandas as pd
import json

In [31]:
# reading files
with open('data/transcripts/transcripts.json', 'r') as infile:
    transcripts = json.load(infile)

to_fill = pd.read_csv('data/to_fill.csv')
# lists that are going to be added to the dataframe
body = []
start = []
end = []

for _, record in to_fill.iterrows():
    # extract the news ID and its respective transcript
    transcript_id = str(record['source_video_id'])
    text = transcripts[transcript_id]['text']
    words = transcripts[transcript_id]['words']

    # calculate the indices of the beginning and end of the news
    body_start = text.find(record['first_words'])
    body_end = text.find(record['last_words']) + len(record['last_words'])

    # append the news body to the list
    body.append(text[body_start:body_end])

    # find the index of the first word of the body
    start_word_index = len(text[:body_start].split(' ')) - 1
    end_word_index = len(text[:body_end].split(' ')) - 1

    # append their start and finish time to respective lists
    start.append(words[start_word_index]['start'])
    end.append(words[start_word_index]['end'])

In [39]:
# add each feature to the dataframe
to_fill['body'] = body
to_fill['start'] = start
to_fill['end'] = end